In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import *
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
extract_size = 1024
class_size = 61

# 图片格式转化
transform_test = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
encoder = EncoderCut()
encoder = encoder.to(device)

test_dataSet = get_test_data(transform_test, encoder, device)
torch.save(test_dataSet, './bottle_neck/resnet152_test_cut.pkl')

# test_dataSet = torch.load('./bottle_neck/resnet152_test_cut.pkl')

encoding 4958/4959->99.98%, spent_time:5:10.85

### init model

In [4]:
classify_model = MultiClassify(extract_size, class_size)
classify_model = classify_model.to(device)

### Load the best trained model,yet!

In [5]:
classify_model.load_state_dict(torch.load('./models/class_multify_rm1layer_good.pkl'))

### How good is the model?

In [6]:
classify_model = classify_model.eval()
predict = []
imgs_ids = []
for embed, img_id in test_dataSet:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    imgs_ids.append(img_id)

In [7]:
df = pd.DataFrame({'image_id': imgs_ids,
                   'disease_class': predict})
df.head()

,disease_class,image_id
0,23,9cd1e21e2d83ec90910ed7796d31d490.jpg
1,24,IMG_20180623_181319_1.jpg
2,45,12bd98dd14260d3ed0eec183eb0c3b9a.jpg
3,46,138dbeb4e9980b92662475024afebbe1.jpg
4,35,f6af0bfb6b3f38263ade0154d8a87466.jpg


In [8]:
df.to_json('test_result/testA_0923.json', orient="records")